# Chronicle of the Trisolarians

<img src="images/trisolarian_image1.jpeg">

This project is inspired by the science fiction "The Three Body Problem" by Liu Cixin.

In the novel, the Trisolarians is a civilization on a planet in an unstable three-star system. As the dynamics of 
the chaotic 4-body system evolves, the planet's climate shifts between unpredictable periods of extreme cold and 
heat -- the so-called Chaotic Era. Moreover, should the planet pass too closely by a star, it may be torn apart by 
tidal force, or worse, completely engulfed by the star. In the book, trisolarian life and civilization are born, 
destroyed, and reborn for many times, before the eventual death of the planet.

In this project, I create a simulation to output a chronicle of the trisolarian planet and its civilization (if 
there is any) given an initial condition for the unstable three-star system.

This is a midterm report of the project.

Xinran Song, 2022/01/30

## 1. Four-Body Orbit Simulation
First, we simulate the orbits of the planet and the 3 stars. Our outputs are (at all steps):

    a) Position of the four bodies.
    b) Energy of the system.
    c) Distance of each of the 3 stars to planet.

### 1.1 Imports & Settings

In [1]:
# Import libraries
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import display, Image
import time
%matplotlib inline

# Set common figure parameters
newparams = {'figure.figsize': (14, 7), 'axes.grid': False,
             'lines.markersize': 6, 'lines.linewidth': 2,
             'font.size': 15, 'mathtext.fontset': 'stix',
             'font.family': 'STIXGeneral'}
plt.rcParams.update(newparams)

### 1.2 Helper functions

a) **RHS (t, y)**

Calculates the RHS of the EoM.

> Parameters:

        t: dummy parameter to be called in ode solver.
        
        y: array. Vector of length 24 holding the current position and velocity of the three objects 
           in the following manner: 
           y = [x1, y1, z1, x2, y2, z2, x3, y3, z3, x4, y4, z4,
                vx1, vy1, vz1, vx2, vy2, vz2, vx3, vy3, vz3, vx4, vy4, vz4].

> Returns:

        z: array. Vector of length 24 holding the derivative of the current position and velocity 
           (the velocity and acceleration) of the three object in the following manner:
           z = [vx1, vy1, vz1, vx2, vy2, vz2, vx3, vy3, vz3, vx4, vy4, vz4,
                ax1, ay1, az1, ax2, ay2, az2, ax3, ay3, az3, ax4, ay4, az4].


In [2]:
def RHS(t, y):

    z = np.zeros(24)
    
    z[:12] = [y[i] for i in range(12, 24)]
    
    r1, r2, r3, r4 = [y[i: i + 3] for i in range(0, 12, 3)]
    
    r21 = np.linalg.norm(r2 - r1)
    r31 = np.linalg.norm(r3 - r1)
    r41 = np.linalg.norm(r4 - r1)
    r32 = np.linalg.norm(r3 - r2)
    r42 = np.linalg.norm(r4 - r2)
    r43 = np.linalg.norm(r4 - r3)
    
    a1 = G * (m2 * (r2 - r1) / r21 ** 3 + m3 * (r3 - r1) / r31 ** 3 + m4 * (r4 - r1) / r41 ** 3)
    a2 = G * (m1 * (r1 - r2) / r21 ** 3 + m3 * (r3 - r2) / r32 ** 3 + m4 * (r4 - r2) / r42 ** 3)
    a3 = G * (m1 * (r1 - r3) / r31 ** 3 + m2 * (r2 - r3) / r32 ** 3 + m4 * (r4 - r3) / r43 ** 3)
    a4 = G * (m1 * (r1 - r4) / r41 ** 3 + m2 * (r2 - r4) / r42 ** 3 + m3 * (r3 - r4) / r43 ** 3)

    z[12:15] = a1
    z[15:18] = a2
    z[18:21] = a3
    z[21:24] = a4

    return z

b) **ode45 (f, t, y, h)**

Calculate next step of an initial value problem (IVP) of an ODE with a RHS described
    by the RHS function with an order 4 approx. and an order 5 approx.

> Parameters:

        f: function pointer. RHS of the ode.
        t: float. Current time.
        y: float. Current step (position).
        h: float. Step-length.

> Returns:

        q: float. Order 2 approx.
        w: float. Order 3 approx.

In [3]:
def ode45(f,t,y,h):
    
    s1 = f(t, y)
    s2 = f(t + h / 4.0, y + h * s1 / 4.0)
    s3 = f(t + 3.0 * h / 8.0, y + 3.0 * h * s1 / 32.0 + 9.0 * h * s2 / 32.0)
    s4 = f(t + 12.0 * h / 13.0, y + 1932.0 * h * s1 / 2197.0 - 7200.0 * h * s2 / 2197.0 + 7296.0 * h * s3 / 2197.0)
    s5 = f(t + h, y + 439.0 * h * s1 / 216.0 - 8.0 * h * s2 + 3680.0 * h * s3 / 513.0 - 845.0 * h * s4 / 4104.0)
    s6 = f(t + h / 2.0, y - 8.0 * h * s1 / 27.0 + 2 * h * s2 - 3544.0 * h * s3 / 2565 + 1859.0 * h * s4 / 4104.0 - 11.0 * h * s5 / 40.0)
    w = y + h * (25.0 * s1 / 216.0 + 1408.0 * s3 / 2565.0 + 2197.0 * s4 / 4104.0 - s5 / 5.0)
    q = y + h * (16.0 * s1 / 135.0 + 6656.0 * s3 / 12825.0 + 28561.0 * s4 / 56430.0 - 9.0 * s5 / 50.0 + 2.0 * s6 / 55.0)
    
    return w, q

c) **energy (z)**

Calculate the mechanical energies of the four body system, we use this function to verify the accuracy of numerical calculations.

> Parameters:

        z: array. Vector of length 24 holding the current position and velocity of the three objects:
           z = [vx1, vy1, vz1, vx2, vy2, vz2, vx3, vy3, vz3, vx4, vy4, vz4,
                ax1, ay1, az1, ax2, ay2, az2, ax3, ay3, az3, ax4, ay4, az4]

> Returns:

        U: float. The potential energies of the system.
        K: float. The kinetic energies of the system.

In [4]:
def energy(z):

    r1, r2, r3, r4 = [z[i: i + 3] for i in range(0, 12, 3)]
    
    # Pairwise distance between objects
    r21 = np.linalg.norm(r2 - r1)
    r31 = np.linalg.norm(r3 - r1)
    r41 = np.linalg.norm(r4 - r1)
    r32 = np.linalg.norm(r3 - r2)
    r42 = np.linalg.norm(r4 - r2)
    r43 = np.linalg.norm(r4 - r3)
    
    # Calculate potential energies
    # First object is "free", second object is moved from infinity to a distance r21 from the first object.
    # Third object is affected gravitationally by both objects.
    U1 = 0
    U2 = -G * m1 * m2 / r21
    U3 = -G * m1 * m3 / r31 - G * m3 * m2 / r32
    U4 = -G * m4 * m1 / r41 - G * m4 * m2 / r42 - G * m4 * m3 / r43
    U = U1 + U2 + U3 + U4
    
    # Calculate kinetic energies of the three objects
    K1 = 0.5 * m1 * np.linalg.norm(z[12:15]) ** 2
    K2 = 0.5 * m2 * np.linalg.norm(z[15:18]) ** 2
    K3 = 0.5 * m3 * np.linalg.norm(z[18:21]) ** 2
    K4 = 0.5 * m4 * np.linalg.norm(z[21:24]) ** 2
    K = K1 + K2 + K3 + K4
    
    return U, K

### 1.3 Plotting

### 1.4 Animation

### 1.5 Workflow

### 1.6 Verification & Examples